In [ ]:
import numpy as np
import pandas as pd
import random
import statsmodels.api as sm
from statsmodels.tools import eval_measures
from sklearn.metrics import r2_score
# from util import all_variable_names_in_df, train_test_split, RANDOM_SEED
from google.colab import files
import random

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# I copied the data to my main drive and named it "Copy_of_train.csv"
path = "/content/gdrive/My Drive/Copy of pop_songs_data _2000s_on.csv"
#path = "/content/gdrive/My Drive/SMAK Data Science/train.csv"
with open(path, 'rb') as f:
  df = pd.read_csv(f)
print("Finished reading data!")

Mounted at /content/gdrive
Finished reading data!


In [ ]:
RANDOM_SEED = 0

def train_test_split(df, train_pct=0.8):
    """
    Input:
        - df: Pandas DataFrame
        - train_pct: optional, float
    Output:
        - train dataframe: Pandas DataFrame
        - test dataframe: Pandas DataFrame
    """
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    msk = np.random.rand(len(df)) < train_pct
    return df[msk], df[~msk]

In [ ]:
train_data, test_data = train_test_split(df)

In [ ]:
import statsmodels.api as sm
from statsmodels.tools import eval_measures
from sklearn.metrics import r2_score

IND_VAR_NAMES = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence', 'key']
DEP_VAR_NAME = "peak_rank"

X_train = train_data[IND_VAR_NAMES]
X_test = test_data[IND_VAR_NAMES]

y_train = train_data[DEP_VAR_NAME]
y_test = test_data[DEP_VAR_NAME]

x_train_with_constant = sm.add_constant(X_train)
mod = sm.OLS(y_train, x_train_with_constant)
result = mod.fit()

mse_train = eval_measures.mse(y_train, result.predict(x_train_with_constant))

x_test_with_constant = sm.add_constant(X_test)
mse_test = eval_measures.mse(y_test, result.predict(x_test_with_constant))

# TODO: Calculate the *test* R-squared value (using sklearn's r2_score function)
rsquared_val = r2_score(y_test, result.predict(x_test_with_constant))

# TODO: Print out the summary to see more information as needed
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              peak_rank   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     19.93
Date:                Tue, 27 Jul 2021   Prob (F-statistic):           9.74e-37
Time:                        14:10:25   Log-Likelihood:                -29930.
No. Observations:                6234   AIC:                         5.988e+04
Df Residuals:                    6223   BIC:                         5.996e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               59.1904      5.140  

In [ ]:
def all_variable_names_in_df(variable_names, df):
    """
    Input:
        - variable_names: list of string, of all the variable
                        names to check whether they're in the df or nah
        - df: Pandas DataFrame

    Output:
        - boolean: True if all the variable names are the columns of df,
                    False if not
    """
    columns = set(df.columns)
    for variable_name in variable_names:
        if variable_name not in columns:
            return False
    return True

In [ ]:
def regression(train_df, test_df, ind_var_names: list, dep_var_name: str):
    """
    Implement Linear Regression using StatsModel.

    inputs:
        - train_df: a Pandas DataFrame, containing all the training samples
        - test_df: a Pandas DataFrame, containing all the testing samples
        - ind_var_names: a list of strings of independent variable columns
                        that we want to include in the model
        - dep_var_name: the name of the dependent variable of our model
    

    outpus:
        - mse_train: the mean-squared error of the model (trained on the training
                    data), evaluated on the training dataset
        - mse_test: the mean-squared error of the model (trained on the training
                    data), evaluated on the testing dataset
        - rsquared_val: the r-squared value of the model (trained on the training
                    data), evaluated on the testing dataset
    """
    ## Stencil: Error check whether the input that you provided to the function is correct or not
    # Do not modify
    for df in [train_df, test_df]:
        assert all_variable_names_in_df(ind_var_names + [dep_var_name], df)

    # TODO: Construct X_train, X_test, y_train, y_test from train_df and test_df, where
    # X_train is a numpy array of all the independent variable instances from train_df,
    # y_train is a numpy array of all the dependent variable instances from train_df,
    # and the same applies to X_test and y_test from test_df.
    # Hint: Look up (1) how to select a Pandas DataFrame B with a subset of columns from a given DataFrame A,
    #           and (2) how to use Pandas .to_numpy() function.
    X_train = train_df[ind_var_names].to_numpy()
    X_test = test_df[ind_var_names].to_numpy()
    y_train = train_df[dep_var_name].to_numpy()
    y_test = test_df[dep_var_name].to_numpy()

    # TODO: Using statsmodel, fit a linear regression model to the training dataset
    # You may checkout statsmodel's documentation here: https://www.statsmodels.org/stable/regression.html
    x_train_with_constant = sm.add_constant(X_train) ##TODO: Why???
    mod = sm.OLS(y_train, x_train_with_constant)
    result = mod.fit()

    # TODO: Using statsmodel's eval_measures MSE calculation function,
    # calculate the Mean-squared Error of the model above (on the training dataset)
    mse_train = eval_measures.mse(y_train, result.predict(x_train_with_constant)) 

    # TODO: Similarly, calculate the Mean-squared Error of the model above (on the testing dataset)
    x_test_with_constant = sm.add_constant(X_test) ##TODO: Why???
    mse_test = eval_measures.mse(y_test, result.predict(x_test_with_constant)) # would the axis be 1?
    
    # TODO: Calculate the *test* R-squared value (using sklearn's r2_score function)
    rsquared_val = r2_score(y_test, result.predict(x_test_with_constant))

    # TODO: Print out the summary to see more information as needed
    print(result.summary())

    # TODO: Replace these values with whatever you found!
    mse_train, mse_test, rsquared_val = mse_train, mse_test, rsquared_val
    
    # And return them! :)
    return mse_train, mse_test, rsquared_val
    

def main():
    # TODO: Load the data from the bike-sharing.csv file into a Pandas DataFrame. Do not change
    # the variable name /data/
    # Hint: Look at the Pandas' read_csv function
    ###data = pd.read_csv("../data/bike-sharing.csv")
   

    
    
    # TODO: Uncomment this to print out the column names of data to know what features there are 
    # in the dataset
    #print("Columns: ", data.columns)

    # TODO: Modify the list IND_VAR_NAMES to select the independent variables you want to perform
    # a linear regression on
    IND_VAR_NAMES = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'loudness', 'speechiness', 'tempo', 'valence', 'key']
    #IND_VAR_NAMES = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'loudness', 'speechiness']
    #IND_VAR_NAMES = ['speechiness']
    

    # Select the dependent variable name DEP_VAR_NAME (default is "cnt")
    DEP_VAR_NAME = "peak_rank"

    # TODO: Using the imported train_test_split function (from util.py), create the train_df and
    # test_df that will be passed into regression. $$ASK HERE!!
    ####train_data, test_data = train_test_split(data)
    #x_train, x_test, y_train, y_test = train_test_split(x, y) ???

    # TODO: Call regression and perform other calculations as you deem necessary to answer the
    # questions posed for this section.
    regression(train_data, test_data, IND_VAR_NAMES, DEP_VAR_NAME)

############ DON'T MODIFY BELOW THIS LINE ############
main()
# if __name__ == "__main__":
#     np.random.seed(RANDOM_SEED)
#     random.seed(RANDOM_SEED)
#     main()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     19.93
Date:                Tue, 27 Jul 2021   Prob (F-statistic):           9.74e-37
Time:                        14:11:43   Log-Likelihood:                -29930.
No. Observations:                6234   AIC:                         5.988e+04
Df Residuals:                    6223   BIC:                         5.996e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         59.1904      5.140     11.515      0.0

In [ ]:
import seaborn as sns


cols2 = ['date', 'weeks_on_board', 'speechiness']


def make_all_data_scatter_plot2(metric):
  scatter_plot = sns.scatterplot()

def make_scatter_plot():
  scatter_plot = sns.catplot(data=train_data.loc[train_data["date"] >= 1990],
  # scatter_plot = sns.catplot(data=train_data,
            kind="strip",
            x="speechiness",
            y= "peak_rank",
            height=5,
            aspect=6,
            hue='date'
            ).set(
                title= 'energy_vs_peak_rank'
            )
  return scatter_plot

make_scatter_plot()

# def make_all_scatterplots2():
#   for col in cols2[1:]:
#     plot = make_scatter_plots2(col)
#     plot.savefig(col + '_output_scatter.png')
#     files.download(col + '_output_scatter.png')
#   print('Done making scatterplots')

# make_all_scatterplots2()